# SSD300 MS COCO Evaluation Tutorial
### Reference: https://github.com/pierluigiferrari/ssd_keras/blob/master/ssd300_evaluation_COCO.ipynb 
- Evaluate a trained SSD300 on one of the MS COCO datasets using the official MS COCO Python tools available here: https://github.com/cocodataset/cocoapi

In [ ]:
from keras import backend as K
from keras.models import load_model
from keras.optimizers import Adam
from scipy.misc import imread
import numpy as np
from matplotlib import pyplot as plt
import sys

# TODO: Specify the directory that contains the `pycocotools` here.
pycocotools_dir = '/datasets/coco_2017/cocoapi-master/PythonAPI/'
if pycocotools_dir not in sys.path:
    sys.path.insert(0, pycocotools_dir)

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from keras_ssd300 import ssd_300
from keras_ssd_loss import SSDLoss
from keras_layer_AnchorBoxes import AnchorBoxes
from keras_layer_L2Normalization import L2Normalization
from ssd_batch_generator import BatchGenerator
from coco_utils import get_coco_category_maps, predict_all_to_json

%matplotlib inline

In [ ]:
# Set the input image size for the model.
img_height = 300
img_width = 300

# 1. Load a trained SSD

## 1.1. Load a trained model

In [ ]:
model_path = 'model/ssd300_coco_2017.h5'

# We need to create an SSDLoss object in order to pass that to the model loader.
ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

K.clear_session() # Clear previous models from memory.

model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                               'L2Normalization': L2Normalization,
                                               'compute_loss': ssd_loss.compute_loss})

# 2. Create a data generator for the evaluation dataset

In [ ]:
dataset = BatchGenerator(box_output_format=['class_id', 'xmin', 'ymin', 'xmax', 'ymax'])

# TODO: Set the paths to the dataset here.
MS_COCO_dataset_images_dir = '/datasets/coco_2017/val2017/'
MS_COCO_dataset_annotations_filename = '/datasets/coco_2017/annotations/instances_val2017.json'

dataset.parse_json(images_dirs=[MS_COCO_dataset_images_dir],
                   annotations_filenames=[MS_COCO_dataset_annotations_filename],
                   ground_truth_available=False, # It doesn't matter whether you set this `True` or `False` because the ground truth won't be used anyway, but the parsing goes faster if you don't load the ground truth.
                   include_classes='all',
                   ret=False)

# We need the `classes_to_cats` dictionary. Read the documentation of this function to understand why.
cats_to_classes, classes_to_cats, cats_to_names, classes_to_names = get_coco_category_maps(MS_COCO_dataset_annotations_filename)

# 3. Run the predictions over the evaluation dataset

In [ ]:
# TODO: Set the desired output file name and the batch size.
results_file = 'detections_val2017_ssd300_results.json'
batch_size = 20 # Ideally, choose a batch size that divides the number of images in the dataset.

In [ ]:
predict_all_to_json(out_file=results_file,
                    model=model,
                    img_height=img_height,
                    img_width=img_width,
                    classes_to_cats=classes_to_cats,
                    batch_generator=dataset,
                    batch_size=batch_size,
                    batch_generator_mode='resize',
                    confidence_thresh=0.01,
                    iou_threshold=0.45,
                    top_k=200,
                    pred_coords='centroids',
                    normalize_coords=True)

# 4. Run the evaluation

In [ ]:
coco_gt   = COCO(MS_COCO_dataset_annotations_filename)
coco_dt   = coco_gt.loadRes(results_file)
image_ids = sorted(coco_gt.getImgIds())

In [ ]:
cocoEval = COCOeval(cocoGt=coco_gt,
                    cocoDt=coco_dt,
                    iouType='bbox')
cocoEval.params.imgIds  = image_ids
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()